In [25]:
import numpy as np

In [26]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [27]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [28]:
def rnn_cell_forward_self(xt, a_prev, parameters):
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    Wya = parameters['Wya']
    ba = parameters['ba']
    by = parameters['by']

    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    yt_pred = softmax(np.dot(Wya, a_next) + by)

    cache = (a_next, a_prev, xt, parameters)

    return a_next, yt_pred, cache

In [29]:
def rnn_forward_self(x, a0, parameters):
    caches = []

    n_x, m, T_x  = x.shape
    n_y, n_a = parameters['Wya'].shape

    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))

    a_next = a0

    for t in range(T_x):
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        a[:,:,t] = a_next
        y_pred[:,:,t] = yt_pred
        caches.append(cache)
    
    caches = (caches, x)# for back-propagation
    return a, y_pred, caches

In [30]:
#LSTM
def lstm_cell_forward_self(xt, a_prev, c_prev, parameters):
    Wf = parameters['Wf'] #forget gate weight
    bf = parameters['bf']

    Wi = parameters['Wi'] #update gate weight
    bi = parameters['bi']

    Wc = parameters['Wc'] #candidate weight
    bc = parameters['bc']

    Wo = parameters['Wo'] #output gate weight
    bo = parameters['bo']

    Wy = parameters['Wy'] #prediction weight
    by = parameters['by']

    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    concat = np.concatenate([a_prev, xt])

    ft = sigmoid(np.dot(Wf, concat) + bf) # Forget gate
    it = sigmoid(np.dot(Wi, concat) + bi) # Update gate
    cct = np.tanh(np.dot(Wc, concat) + bc) # Candidate value
    c_next = c_prev * ft + cct * it
    ot = sigmoid(np.dot(Wo, concat) + bo) # Output gate
    a_next = ot * (np.tanh(c_next))

    yt_pred = softmax(np.dot(Wy, a_next) + by)

    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

In [31]:
def lstm_forward_self(x, a0, parameters):
    caches = []

    n_x, m, T_x = x.shape
    n_y, n_a = parameters['Wy'].shape

    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeors((n_y, m , T_x))

    a_next = a0
    c_next = np.zeros((n_a, m))

    for t in range(T_x):
        xt = x[:,:,t]

        a_next, c_next, yt, cache = lstm_cell_forward(xt, a_next, c_next, parameters)

        a[:,:,t] = a_next
        c[:,:,t] = c_next
        y[:,:,t] = yt

        caches.append(cache)

    caches = (caches, x) # for back-propagation
    return a, y, c, caches